In [1]:
import requests, json
import pandas as pd
from pprint import pprint

pd.set_option('display.max_columns', None)

In [2]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 6796212}


In [3]:
# create players dataframe
players = pd.json_normalize(r['elements'])

# show some information about first five players
players[['id', 'web_name', 'team', 'element_type']].head()

,id,web_name,team,element_type
0,1,Cédric,1,2
1,3,Xhaka,1,3
2,4,Elneny,1,3
3,5,Holding,1,2
4,6,Partey,1,3


In [4]:
# create teams dataframe
teams = pd.json_normalize(r['teams'])

teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1200,1270,1150,1210,1190,1220,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1090,1100,1140,1110,1090,1090,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,2,None,False,0,1050,1050,1080,1130,1060,1100,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,3,None,False,0,1090,1100,1120,1150,1080,1120,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,3,None,False,0,1100,1090,1160,1160,1100,1120,131


In [5]:
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],55
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],206
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],237
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],60


In [6]:
# join players to teams
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)

# show joined result
df[['first_name', 'second_name', 'name']].head()

,first_name,second_name,name
0,Cédric,Alves Soares,Arsenal
1,Granit,Xhaka,Arsenal
2,Mohamed,Elneny,Arsenal
3,Rob,Holding,Arsenal
4,Thomas,Partey,Arsenal


In [7]:
# join player positions
df = df.merge(
    positions,
    left_on='element_type',
    right_on='id'
)

# rename columns
df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)

# show result
df[
    ['first_name', 'second_name', 'team_name', 'position_name']
].head()

,first_name,second_name,team_name,position_name
0,Cédric,Alves Soares,Arsenal,Defender
1,Rob,Holding,Arsenal,Defender
2,Kieran,Tierney,Arsenal,Defender
3,Benjamin,White,Arsenal,Defender
4,Takehiro,Tomiyasu,Arsenal,Defender


In [8]:
# get data from 'element-summary/{PID}/' endpoint for PID=4
r = requests.get(base_url + 'element-summary/4/').json()

# show top-level fields for player summary
pprint(r, depth=1)


{'fixtures': [...], 'history': [], 'history_past': [...]}


In [9]:
df[
    ['id_x', 'first_name', 'second_name', 'team_name', 'position_name']
].head()


,id_x,first_name,second_name,team_name,position_name
0,1,Cédric,Alves Soares,Arsenal,Defender
1,5,Rob,Holding,Arsenal,Defender
2,8,Kieran,Tierney,Arsenal,Defender
3,10,Benjamin,White,Arsenal,Defender
4,14,Takehiro,Tomiyasu,Arsenal,Defender


In [64]:
i=0
for id_x in df['id_x']:
    file_path_name = './data/player_season_history/' + str(id_x) + '.json'
    player = df[df['id_x']==id_x]
    player_name = player['first_name'][i] + ' ' + player['second_name'][i]
    uri = base_url + 'element-summary/' + str(id_x) + '/'
    id_x_resp = requests.get(base_url + 'element-summary/' + str(id_x) + '/').json()
    print(f"{id_x} Saving historical season data for {player_name}.") #.format(mean_legendary_hp - mean_not_legendary_hp))
    dfr = pd.DataFrame(id_x_resp['history_past'])
    dfr.insert(0, 'second_name', [player['second_name'][i] for j in range(len(dfr))])
    dfr.insert(0, 'first_name', [player['first_name'][i] for j in range(len(dfr))])
    dfr.insert(0, 'player_name', [player_name for j in range(len(dfr))])
    result = dfr.to_json(orient='records')
    parsed = json.loads(result)
    json_str = json.dumps(parsed, indent=2)
    print(json_str)
    f = open(file_path_name, 'w')
    f.write(json_str)
    f.close()
    i+=1
    if i==3:
        break


1 Saving historical season data for Cédric Alves Soares.


,player_name,first_name,second_name,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index
0,Cédric Alves Soares,Cédric,Alves Soares,2015/16,58822,50,47,86,1965,0,2,9,19,0,0,0,3,0,0,7,506,0.0,0.0,0.0,0.0
1,Cédric Alves Soares,Cédric,Alves Soares,2016/17,58822,50,49,102,2515,0,3,11,36,0,0,0,7,0,0,11,584,591.0,648.9,155.0,139.6
2,Cédric Alves Soares,Cédric,Alves Soares,2017/18,58822,50,47,85,2794,0,3,7,44,0,0,0,3,0,0,2,481,507.0,455.6,159.0,112.3
3,Cédric Alves Soares,Cédric,Alves Soares,2018/19,58822,45,42,52,1493,1,2,4,31,0,0,0,4,0,0,7,273,309.0,226.5,103.0,63.9
4,Cédric Alves Soares,Cédric,Alves Soares,2019/20,58822,50,48,61,1553,1,1,4,20,0,0,0,1,0,0,3,286,349.0,218.9,118.0,68.7
5,Cédric Alves Soares,Cédric,Alves Soares,2020/21,58822,50,46,28,744,0,1,2,11,0,0,0,1,0,0,3,125,110.8,114.8,66.0,29.2
6,Cédric Alves Soares,Cédric,Alves Soares,2021/22,58822,45,42,48,1481,1,1,3,27,0,0,0,3,0,0,3,292,318.4,327.1,111.0,75.8


5 Saving historical season data for Rob Holding.


,player_name,first_name,second_name,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index
0,Rob Holding,Rob,Holding,2016/17,156074,45,41,32,810,0,0,4,8,0,0,0,3,0,0,3,171,172.0,49.2,100.0,32.2
1,Rob Holding,Rob,Holding,2017/18,156074,50,44,27,819,0,1,3,17,0,0,0,3,0,0,1,196,212.2,39.3,24.0,27.5
2,Rob Holding,Rob,Holding,2018/19,156074,45,43,24,806,0,0,1,10,0,0,0,1,0,0,5,148,187.8,28.7,14.0,23.2
3,Rob Holding,Rob,Holding,2019/20,156074,45,44,16,580,0,0,1,9,0,0,0,1,0,0,2,96,133.6,4.8,19.0,15.8
4,Rob Holding,Rob,Holding,2020/21,156074,45,43,105,2557,0,1,11,26,1,0,0,2,0,0,10,536,602.6,76.1,97.0,77.7
5,Rob Holding,Rob,Holding,2021/22,156074,45,41,29,840,1,0,2,16,0,0,0,2,1,0,2,152,253.0,8.6,49.0,31.1


8 Saving historical season data for Kieran Tierney.


,player_name,first_name,second_name,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index
0,Kieran Tierney,Kieran,Tierney,2019/20,192895,55,53,42,986,1,2,2,16,0,0,0,2,0,0,4,213,201.8,121.1,73.0,39.8
1,Kieran Tierney,Kieran,Tierney,2020/21,192895,55,52,96,2299,1,4,8,27,0,0,0,4,0,0,4,483,446.4,490.1,201.0,113.9
2,Kieran Tierney,Kieran,Tierney,2021/22,192895,50,49,106,1916,1,3,11,21,0,0,0,0,0,0,11,454,365.2,335.9,198.0,90.1
